# Data cleaning

This notebook aims to outline the initial data preparation before building a price prediction model.
The data seen in this notebook was scraped from the autotrader website. This notebook will do the following:

- [Load the vehicle features and sellers tables from the database](#read_data)
- [Assess dataset](#assess_data)
- [Outline the cleaning process prior to EDA](#clean_data)
- [Output a cleaner dataset for EDA](#pickle_data)

In [1]:
import pandas as pd
import mysql.connector
from autotrader_scraper.autotrader_scraper.config import mysql_details

## Read data tables into pandas <a id='read_data'></a>

Connect to database and join vehicle features with sellers infomation. 

In [2]:
DB_NAME = 'autotrader_adverts'

cnx = mysql.connector.connect(**mysql_details)
cursor = cnx.cursor(dictionary=True)

cursor.execute("USE {}".format(DB_NAME))
cursor.execute('''SELECT * 
                  FROM vehicle_features as vf
                  LEFT JOIN sellers as s
                  ON vf.seller_id=s.seller_id
                  ORDER BY date_scraped ASC, time_scraped ASC''')
full_results = cursor.fetchall()
cnx.close()

In [3]:
full_df = pd.DataFrame(full_results)
full_df.head(10)

,advert_id,date_scraped,time_scraped,make,model,trim,manufactured_year,manufactured_year_identifier,body_type,mileage,...,total_reviews,region,county,town,country,seller_postcode,seller_address_one,seller_address_two,dealer_website,primary_contact_number
0,202107165101873,2021-08-22,0 days 00:55:25,DS AUTOMOBILES,DS 3,Prestige,2018.0,18,Hatchback,31537.0,...,13067.0,SOUTH EAST,KENT,ADDINGTON,GB,ME19 5PL,A20 London Road,None,https://www.bigmotoringworld.co.uk/autotraderv...,01634215708
1,202009113616600,2021-08-22,0 days 00:55:29,Vauxhall,Astra,SXi,2007.0,07,Hatchback,70000.0,...,18.0,LONDON,MIDDLESEX,HOUNSLOW,GB,TW4 6JQ,"VISTA BUSINESS CENTRE, SALISBURY ROAD",None,https://www.motorpedia.uk/,02080337311
2,202107305627204,2021-08-22,0 days 00:55:29,Volkswagen,Polo,Moda,2010.0,60,Hatchback,89000.0,...,125.0,LONDON,HERTFORDSHIRE,BARNET,GB,EN5 4RY,BENTLEY HEATH LANE,None,http://www.mynextcar.co.uk,02081152043
3,202108206481776,2021-08-22,0 days 00:55:30,Vauxhall,Astra,SXi,2008.0,08,Hatchback,117000.0,...,7.0,LONDON,ESSEX,ILFORD,GB,IG3 8RW,777-779 High Road,Seven Kings,https://dmsgateway.autotrader.co.uk/api/advert...,07441907724
4,202108206471965,2021-08-22,0 days 00:55:30,Mazda,Mazda3,Sport,2007.0,07,Hatchback,93500.0,...,18.0,SOUTH WEST,WILTSHIRE,SWINDON,GB,SN1 2PG,"UNIT 10-11, ISIS TRADING ESTATE",None,https://dmsgateway.autotrader.co.uk/api/advert...,07537125171
5,202108136200337,2021-08-22,0 days 00:55:30,Vauxhall,Astra,Club,2004.0,54,Hatchback,73888.0,...,10.0,SOUTH EAST,SURREY,WORCESTER PARK,None,None,None,None,None,07971223786
6,202108045818203,2021-08-22,0 days 00:55:30,Volvo,S40,SE,2005.0,55,Saloon,160000.0,...,15.0,LONDON,MIDDLESEX,LONDON,GB,None,None,None,https://dmsgateway.autotrader.co.uk/api/advert...,07537165960
7,202108206446218,2021-08-22,0 days 00:55:31,Ford,Fiesta,Ghia,2006.0,06,Hatchback,63000.0,...,13.0,SOUTH EAST,EAST SUSSEX,UCKFIELD,None,None,None,None,None,07971243842
8,202108206442132,2021-08-22,0 days 00:55:31,MINI,Hatch,Cooper,2004.0,04,Hatchback,63977.0,...,37.0,LONDON,SURREY,WANDSWORTH,GB,SW18 4QA,"152-156, Penwith Road",None,None,07537165675
9,202105243041206,2021-08-22,0 days 00:55:31,Vauxhall,Corsa,SE,2009.0,09,Hatchback,100843.0,...,128.0,NORTH WEST,CHESHIRE,NORTHWICH,GB,CW8 1BE,Unit 5 RIVERSIDE TRADING ESTATE,NAVIGATION ROAD,https://dmsgateway.autotrader.co.uk/api/advert...,07537164943


## Assess data <a id='assess_data'></a>

In [4]:
full_df.shape

(27808, 77)

Some of the columns contain nulls and some of the data types are incorrect.

In [5]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27808 entries, 0 to 27807
Data columns (total 77 columns):
 #   Column                        Non-Null Count  Dtype          
---  ------                        --------------  -----          
 0   advert_id                     27808 non-null  int64          
 1   date_scraped                  27808 non-null  object         
 2   time_scraped                  27808 non-null  timedelta64[ns]
 3   make                          27808 non-null  object         
 4   model                         27808 non-null  object         
 5   trim                          26792 non-null  object         
 6   manufactured_year             27787 non-null  float64        
 7   manufactured_year_identifier  27508 non-null  object         
 8   body_type                     27790 non-null  object         
 9   mileage                       27761 non-null  float64        
 10  engine_size                   27565 non-null  object         
 11  transmission   

Percentage of missing values in each column in descending order.

In [6]:
print(pd.DataFrame((full_df.isnull().sum()/len(full_df))*100).sort_values(0, ascending=False).to_string())

                                      0
max_loading_weight            82.771145
zero_to_sixty_two             66.218354
gross_vehicle_weight          65.330121
seller_address_two            61.388809
number_of_owners              45.857307
price_deviation               35.399885
price_deviation_type          35.399885
price_rating                  35.399885
price_rating_label            35.399885
boot_space_seats_down         35.209292
emission_scheme               33.076812
zero_to_sixty                 29.433976
urban                         25.722814
extra_urban                   25.672468
seller_rating                 22.292146
total_reviews                 22.292146
dealer_website                21.364356
minimum_kerb_weight           20.386220
vehicle_location_postcode     17.599252
vehicle_location_latitude     17.599252
vehicle_location_longitude    17.599252
seller_postcode               16.369390
seller_address_one            16.369390
seller_longlat                16.369390


Checking duplicate entries

In [7]:
full_df['advert_id'].duplicated().sum()

0

In [8]:
full_df['country'].unique()

array(['GB', None], dtype=object)

In [9]:
full_df['is_dealer_trusted'].unique()

array([ 0., nan])

In [10]:
full_df['car_condition'].unique()

array(['Used'], dtype=object)

## Clean Data <a id='explore_data'></a>

The columns which will not be used for the price prediction model will be dropped.
The following columns will be dropped as they uniquely identify each car ad and will lead to overfitting in the model:
- advert_id
- derivative_id
- vehicle_registration_mark 
- seller_id (not uniquely but still identifies car ads)
- seller_name (not uniquely but still identifies car ads)

The following columns are not available at the time predictions are made and can lead to data leaks:
- price_deviation
- price_deviation_type
- price_excluding_fees
- no_admin_fees
- price_rating
- price_rating_label

The following columns have a lot of data points missing:
- max_loading_weight
- gross_vehicle_weight
- zero_to_sixty_two
- zero_to_sixty
- minimum_kerb_weight

The following columns are not useful as they only have one category:
- car_condition (all 'used')
- is_dealer_trusted (all False)
- country (all 'GB')

The following columns will probably not be a good predictor of price:
- ad_description
- page_url
- primary_contact_number
- dealer_website
- date_scraped
- time_scraped

The infomation in the following columns is already represented by another column:
- manufactured_year_identifier (year)
- seller_address_one (lat and long)
- seller_address_two (lat and long)
- vehicle_location_postcode (lat and long)
- seller_postcode (lat and long)

The following columns contain inaccurate data:
- average_mileage (average based on full dataset on website but only a fraction was scraped)
- mileage_deviation
- mileage_deviation_type 


In [11]:
columns_to_drop = ['max_loading_weight', 'zero_to_sixty_two', 'gross_vehicle_weight',   
'seller_address_two', 'price_deviation', 'price_deviation_type',  
'price_rating', 'price_rating_label', 'zero_to_sixty', 'dealer_website',               
'primary_contact_number', 'seller_name', 'seller_id', 'advert_id', 'date_scraped', 'time_scraped',
'vehicle_location_postcode', 'seller_postcode', 'seller_address_one', 'page_url', 'country',
'ad_description', 'price_excluding_fees', 'no_admin_fees', 'is_dealer_trusted', 'manufactured_year_identifier',
'vehicle_registration_mark', 'derivative_id', 'car_condition', 'minimum_kerb_weight', 'average_mileage',
'mileage_deviation', 'mileage_deviation_type']

df = full_df.drop(columns_to_drop, axis=1)

In [12]:
df.columns

Index(['make', 'model', 'trim', 'manufactured_year', 'body_type', 'mileage',
       'engine_size', 'transmission', 'fuel_type', 'doors', 'seats',
       'number_of_owners', 'emission_scheme', 'vehicle_location_latitude',
       'vehicle_location_longitude', 'imported', 'price', 'number_of_photos',
       'co2_emission', 'tax', 'top_speed', 'cylinders', 'valves',
       'engine_power', 'engine_torque', 'height', 'length', 'wheelbase',
       'width', 'fuel_tank_capacity', 'boot_space_seats_up',
       'boot_space_seats_down', 'urban', 'extra_urban', 'combined',
       'co2_emissions', 'insurance_group', 'seller_longlat', 'seller_segment',
       'seller_rating', 'total_reviews', 'region', 'county', 'town'],
      dtype='object')

### Cleaning issues

- vehicle_location_longitude & vehicle_location_latitude contains the same data as seller_longlat but has more missing values
- there are two columns with CO2 emissions
- manufactured_year, doors, seats, engine_power, wheelbase, height, legnth, valves, cylinders, top_speed, tax, CO2 emissions are all floats
- make, model and trim are interlinked so could put into one column
- the emission_scheme column only has one 'ULEZ' category (convert to boolean column)

Combine latitude and longitudes into one column.

In [13]:
df2 = df.copy()

In [14]:
df2[(df2.vehicle_location_longitude.isnull()) & (~df2.seller_longlat.isnull())][['vehicle_location_longitude', 'vehicle_location_latitude', 'seller_longlat']]

,vehicle_location_longitude,vehicle_location_latitude,seller_longlat
841,None,None,"51.631083,-0.761748"
1204,None,None,"51.5283026,-0.1302148"
1209,None,None,"53.7201316,-0.4299832"
1225,None,None,"52.9229051,-1.4600297"
1358,None,None,"51.5283026,-0.1302148"
...,...,...,...
27097,None,None,"51.5283026,-0.1302148"
27101,None,None,"51.5283026,-0.1302148"
27193,None,None,"51.5283026,-0.1302148"
27430,None,None,"51.5283026,-0.1302148"


In [15]:
latitudes = df2[(df2.vehicle_location_longitude.isnull()) & (~df2.seller_longlat.isnull())].seller_longlat.map(lambda x: x.split(',')[0])
longitudes = df2[(df2.vehicle_location_longitude.isnull()) & (~df2.seller_longlat.isnull())].seller_longlat.map(lambda x: x.split(',')[1])

In [16]:
for index, value in latitudes.items():
    df2.loc[index, 'vehicle_location_latitude'] = value

In [17]:
for index, value in longitudes.items():
    df2.loc[index, 'vehicle_location_longitude'] = value

In [18]:
df2[(df2.vehicle_location_longitude.isnull()) & (~df2.seller_longlat.isnull())]

,make,model,trim,manufactured_year,body_type,mileage,engine_size,transmission,fuel_type,doors,...,combined,co2_emissions,insurance_group,seller_longlat,seller_segment,seller_rating,total_reviews,region,county,town


In [19]:
df2[(df2.vehicle_location_latitude.isnull()) & (~df2.seller_longlat.isnull())]

,make,model,trim,manufactured_year,body_type,mileage,engine_size,transmission,fuel_type,doors,...,combined,co2_emissions,insurance_group,seller_longlat,seller_segment,seller_rating,total_reviews,region,county,town


In [20]:
df2.drop('seller_longlat', axis=1, inplace=True)

Combine CO2 emissions into one column.

In [21]:
df2[(df2.co2_emissions.isnull()) & (~df2.co2_emission.isnull())][['co2_emission', 'co2_emissions']]

,co2_emission,co2_emissions
74,220.0,NaN
7896,142.0,NaN
22593,258.0,NaN


In [22]:
co2_emissions = df2[(df2.co2_emissions.isnull()) & (~df2.co2_emission.isnull())].co2_emission

In [23]:
for index, value in co2_emissions.items():
    df2.loc[index, 'co2_emissions'] = value

In [24]:
df2[(df2.co2_emissions.isnull()) & (~df2.co2_emission.isnull())]

,make,model,trim,manufactured_year,body_type,mileage,engine_size,transmission,fuel_type,doors,...,extra_urban,combined,co2_emissions,insurance_group,seller_segment,seller_rating,total_reviews,region,county,town


In [25]:
df2.drop('co2_emission', axis=1, inplace=True)

Convert emission_scheme column to boolean to show if a car is ULEZ compliant or not.

In [26]:
df2['emission_scheme'].unique()

array(['ULEZ', None], dtype=object)

In [27]:
def ULEZ_boolean(x):
    '''
    Returns a 1 if 'ULEZ' string is given or a 0 if not.
    '''
    
    if x == 'ULEZ':
        x = 1
    else:
        x = 0
    return x

In [28]:
df2['ulez'] = df2['emission_scheme'].map(ULEZ_boolean)

In [29]:
df2.drop('emission_scheme', axis=1, inplace=True)

Fix datatypes of columns, and make data entries lowercase

In [30]:
df2 = df2.astype(dtype={'manufactured_year': 'Int64', 'mileage': 'Int64',
                        'doors': 'Int64', 'seats': 'Int64',
                        'number_of_owners': 'Int64', 'tax': 'Int64',
                        'top_speed': 'Int64', 'cylinders': 'Int64',
                        'valves': 'Int64', 'engine_power': 'Int64',
                        'height': 'Int64', 'length': 'Int64',
                        'wheelbase': 'Int64', 'width': 'Int64',
                        'boot_space_seats_up': 'Int64', 'boot_space_seats_down': 'Int64',
                        'co2_emissions': 'Int64', 'total_reviews': 'Int64',
                        'engine_size': 'float', 'vehicle_location_latitude': 'float',
                        'vehicle_location_longitude': 'float', 'engine_torque': 'float',
                        'fuel_tank_capacity': 'float', 'urban': 'float', 'extra_urban': 'float',
                        'combined': 'float', 'seller_rating': 'float'
                        })

In [31]:
df2['make'] = df2.make.map(lambda x: x.strip().lower().replace(' ', '-'), na_action='ignore')
df2['model'] = df2.model.map(lambda x: x.strip().lower().replace(' ', '-'), na_action='ignore')
df2['trim'] = df2.trim.map(lambda x: x.strip().lower().replace(' ', '-'), na_action='ignore')
df2['body_type'] = df2.body_type.map(lambda x: x.strip().lower().replace(' ', '_'), na_action='ignore').astype('category')
df2['transmission'] = df2.transmission.map(lambda x: x.strip().lower().replace(' ', '_'), na_action='ignore').astype('category')
df2['fuel_type'] = df2.fuel_type.map(lambda x: x.strip().lower().replace(' ', '_'), na_action='ignore').astype('category')
df2['insurance_group'] = df2.insurance_group.map(lambda x: x.strip().lower().replace(' ', '_'), na_action='ignore').astype('category')
df2['seller_segment'] = df2.seller_segment.map(lambda x: x.strip().lower().replace(' ', '_'), na_action='ignore').astype('category')
df2['region'] = df2.region.map(lambda x: x.strip().lower().replace(' ', '_'), na_action='ignore').astype('category')
df2['county'] = df2.county.map(lambda x: x.strip().lower().replace(' ', '_'), na_action='ignore').astype('category')
df2['town'] = df2.town.map(lambda x: x.strip().lower().replace(' ', '_'), na_action='ignore').astype('category')

Combine make, model and trim into one column.

In [32]:
df2['make_model_trim'] = df2.make + '_' + df2.model + '_' + df2.trim.fillna('')

In [33]:
df2['make_model_trim'] = df2['make_model_trim'].astype('category')

In [34]:
df2.drop(['make', 'model', 'trim'], axis=1, inplace=True)

In [35]:
df2.columns

Index(['manufactured_year', 'body_type', 'mileage', 'engine_size',
       'transmission', 'fuel_type', 'doors', 'seats', 'number_of_owners',
       'vehicle_location_latitude', 'vehicle_location_longitude', 'imported',
       'price', 'number_of_photos', 'tax', 'top_speed', 'cylinders', 'valves',
       'engine_power', 'engine_torque', 'height', 'length', 'wheelbase',
       'width', 'fuel_tank_capacity', 'boot_space_seats_up',
       'boot_space_seats_down', 'urban', 'extra_urban', 'combined',
       'co2_emissions', 'insurance_group', 'seller_segment', 'seller_rating',
       'total_reviews', 'region', 'county', 'town', 'ulez', 'make_model_trim'],
      dtype='object')

In [36]:
df2 = df2[['make_model_trim', 'manufactured_year', 'body_type', 'mileage', 'engine_size',
       'transmission', 'fuel_type', 'doors', 'seats', 'number_of_owners',
       'vehicle_location_latitude', 'vehicle_location_longitude', 'imported',
       'price', 'number_of_photos', 'tax', 'top_speed', 'cylinders', 'valves',
       'engine_power', 'engine_torque', 'height', 'length', 'wheelbase',
       'width', 'fuel_tank_capacity', 'boot_space_seats_up',
       'boot_space_seats_down', 'urban', 'extra_urban', 'combined',
       'co2_emissions', 'insurance_group', 'seller_segment', 'seller_rating',
       'total_reviews', 'region', 'county', 'town', 'ulez']]

In [37]:
df2.head()

,make_model_trim,manufactured_year,body_type,mileage,engine_size,transmission,fuel_type,doors,seats,number_of_owners,...,combined,co2_emissions,insurance_group,seller_segment,seller_rating,total_reviews,region,county,town,ulez
0,ds-automobiles_ds-3_prestige,2018,hatchback,31537,1.6,manual,diesel,3,5,1,...,78.5,94,26e,independent,4.7,13067,south_east,kent,addington,1
1,vauxhall_astra_sxi,2007,hatchback,70000,1.4,manual,petrol,3,5,<NA>,...,46.3,146,10e,independent,5.0,18,london,middlesex,hounslow,1
2,volkswagen_polo_moda,2010,hatchback,89000,1.2,manual,petrol,3,5,<NA>,...,51.4,128,05e,independent,4.8,125,london,hertfordshire,barnet,1
3,vauxhall_astra_sxi,2008,hatchback,117000,1.4,manual,petrol,5,5,3,...,46.3,146,10e,independent,4.4,7,london,essex,ilford,1
4,mazda_mazda3_sport,2007,hatchback,93500,2.0,manual,petrol,5,5,2,...,35.8,189,20e,independent,5.0,18,south_west,wiltshire,swindon,1


In [38]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27808 entries, 0 to 27807
Data columns (total 40 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   make_model_trim             27808 non-null  category
 1   manufactured_year           27787 non-null  Int64   
 2   body_type                   27790 non-null  category
 3   mileage                     27761 non-null  Int64   
 4   engine_size                 27565 non-null  float64 
 5   transmission                27807 non-null  category
 6   fuel_type                   27807 non-null  category
 7   doors                       27769 non-null  Int64   
 8   seats                       27272 non-null  Int64   
 9   number_of_owners            15056 non-null  Int64   
 10  vehicle_location_latitude   23259 non-null  float64 
 11  vehicle_location_longitude  23259 non-null  float64 
 12  imported                    27808 non-null  int64   
 13  price           

## Output dataset to file <a id='pickle_data'></a>

Save this dataset to a file which will hold the datatypes. There are still nulls in this dataset to preserve as much data as possible 
before performing exploratory data anaylysis. The effect of dropping nulls vs imputing values will be evaluated for the prediction model.

In [39]:
df2.to_pickle("cars_data.pkl")